In [21]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical,plot_model

import pandas as pd
import numpy as np
import imageio
from numpy import random
import cv2
import pafy
import os
%matplotlib inline

In [22]:
import datetime as dt 
from collections import deque
from moviepy.editor import *
import pafy
import random
import time

In [23]:
import create_dataset

In [24]:
#features,labels=create_dataset.create_dataset()

In [25]:
#features=np.array(features)
#np.save('features.npy',features)
#labels=np.array(labels)
#np.save('labels.npy',labels)

In [26]:
seed_constant = 50
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [27]:
labels=np.load('labels.npy')
features=np.load('features.npy')

In [28]:
num_class=len(np.unique(labels))

In [29]:
labels=to_categorical(labels,num_classes=num_class)

In [30]:
 X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [31]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 5

In [32]:
class PositionalEmbedding(layers.Layer):
    def __init__(self,sequence_length,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.position_embedding=layers.Embedding(input_dim=sequence_length,
                                                output_dim=output_dim)
        
        self.sequence_length=sequence_length
        self.output_dim=output_dim
        
    def call(self,inputs):
        length=tf.shape(inputs)[1]
        positions=tf.range(start=0,limit=length,delta=1)
        embedded_positions=self.position_embedding(positions)
        return inputs +embedded_positions
    
    def compute_mask(self,inputs,mask=None):
        mask=tf.reduce_any(tf.cast(inputs,'bool'),axis=-1)
        return mask

In [36]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.4
        )
        self.dense_proj = tf.keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [37]:
##Qn  Why no TransformerDecoder was used
## not used in the paper vivit
## look into vivit paper for more information 


In [38]:
def get_compiled_model():
    sequence_length=MAX_SEQ_LENGTH
    embed_dim=NUM_FEATURES
    dense_dim=4
    num_heads=1
    classes=num_class
    lr=0.001
    inputs=tf.keras.Input(shape=(sequence_length,embed_dim))
    x=PositionalEmbedding(sequence_length,embed_dim,name='Frame_positional_embedding')(inputs)
    x=TransformerEncoder(embed_dim,dense_dim,num_heads,name='TransformerEncoder')(x)
    x=GlobalMaxPool1D()(x)
    x=Dropout(0.5)(x)
    
    outputs=Dense(num_class,activation='softmax')(x)
    model=tf.keras.Model(inputs,outputs,name='vit')
    model.compile(optimizer=Adam(learning_rate=lr),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
    
    

In [39]:
vit_model_no_tubelet=get_compiled_model()

In [40]:
vit_model_no_tubelet.summary()

Model: "vit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20, 1024)]        0         
                                                                 
 Frame_positional_embedding   (None, 20, 1024)         20480     
 (PositionalEmbedding)                                           
                                                                 
 TransformerEncoder (Transfo  (None, 20, 1024)         4211716   
 rmerEncoder)                                                    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 1024)             0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                               

In [41]:
import time
start_time=time.time()
saved_filespath='saved_model_no_tubelet'
checkpoint=tf.keras.callbacks.ModelCheckpoint(saved_filespath,saved_best_only=True,saved_weights_only=True,verbose=1)
vit_model_no_tubelet.fit(x=X_train,y=y_train,validation_split=0.2,verbose=1,epochs=50,shuffle=True,
             callbacks=[checkpoint])
total_time=(time.time()-start_time)/60.0
print('Total eplased time: {:.3f}'.format(total_time))

Epoch 1/50
1/3 [=========>....................] - ETA: 3s - loss: 2.4153 - accuracy: 0.4375
Epoch 1: saving model to saved_model_no_tubelet


INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets


INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets
  layer_config = serialize_layer_fn(layer)

  return generic_utils.serialize_keras_object(obj)



3/3 [==============================] - 7s 2s/step - loss: 4.8490 - accuracy: 0.5455 - val_loss: 1.1761 - val_accuracy: 0.6471
Epoch 2/50
1/3 [=========>....................] - ETA: 0s - loss: 1.0853 - accuracy: 0.7500
Epoch 2: saving model to saved_model_no_tubelet


INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets


...

INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets
  layer_config = serialize_layer_fn(layer)

  return generic_utils.serialize_keras_object(obj)



3/3 [==============================] - 3s 2s/step - loss: 6.9740e-04 - accuracy: 1.0000 - val_loss: 0.6188 - val_accuracy: 0.8824
Epoch 50/50
1/3 [=========>....................] - ETA: 0s - loss: 0.0018 - accuracy: 1.0000
Epoch 50: saving model to saved_model_no_tubelet


INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets


INFO:tensorflow:Assets written to: saved_model_no_tubelet/assets
  layer_config = serialize_layer_fn(layer)

  return generic_utils.serialize_keras_object(obj)



3/3 [==============================] - 4s 2s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.5580 - val_accuracy: 0.9412
Total eplased time: 3.508


In [42]:
vit_model_no_tubelet.evaluate(x=X_test,y=y_test,verbose=1)

2/2 [==============================] - 0s 10ms/step - loss: 0.1835 - accuracy: 0.9762


[0.18348415195941925, 0.976190447807312]